In [22]:
import findspark
findspark.init()

In [23]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [24]:
cpulmon = spark.read.csv("c:/data/lung_cancer.csv",header=True, inferSchema=True, nullValue="NA")

In [25]:
cpulmon = cpulmon.drop('ANXIETY','ALLERGY',)
cpulmon.show(10)

+------+---+-------+--------------+-------------+---------------+--------+--------+--------+-----------------+--------+-------------------+---------------------+----------+-----------+
|GENDER|AGE|SMOKING|YELLOW_FINGERS|PEER_PRESSURE|CHRONIC_DISEASE|FATIGUE |ALLERGY |WHEEZING|ALCOHOL_CONSUMING|COUGHING|SHORTNESS_OF_BREATH|SWALLOWING_DIFFICULTY|CHEST_PAIN|LUNG_CANCER|
+------+---+-------+--------------+-------------+---------------+--------+--------+--------+-----------------+--------+-------------------+---------------------+----------+-----------+
|     M| 69|      1|             2|            1|              1|       2|       1|       2|                2|       2|                  2|                    2|         2|        YES|
|     M| 74|      2|             1|            1|              2|       2|       2|       1|                1|       1|                  2|                    2|         2|        YES|
|     F| 59|      1|             1|            2|              1|       2| 

In [26]:
#Limpieza de datos

cpulmon = cpulmon.filter('SMOKING IS NOT NULL')
cpulmon = cpulmon.filter('AGE IS NOT NULL')
cpulmon = cpulmon.dropna()
cpulmon.show(20)

+------+---+-------+--------------+-------------+---------------+--------+--------+--------+-----------------+--------+-------------------+---------------------+----------+-----------+
|GENDER|AGE|SMOKING|YELLOW_FINGERS|PEER_PRESSURE|CHRONIC_DISEASE|FATIGUE |ALLERGY |WHEEZING|ALCOHOL_CONSUMING|COUGHING|SHORTNESS_OF_BREATH|SWALLOWING_DIFFICULTY|CHEST_PAIN|LUNG_CANCER|
+------+---+-------+--------------+-------------+---------------+--------+--------+--------+-----------------+--------+-------------------+---------------------+----------+-----------+
|     M| 69|      1|             2|            1|              1|       2|       1|       2|                2|       2|                  2|                    2|         2|        YES|
|     M| 74|      2|             1|            1|              2|       2|       2|       1|                1|       1|                  2|                    2|         2|        YES|
|     F| 59|      1|             1|            2|              1|       2| 

In [27]:
from pyspark.ml.feature import StringIndexer

cpulmon = StringIndexer(inputCol='GENDER', outputCol= 'gd_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='SMOKING', outputCol= 'smk_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='YELLOW_FINGERS', outputCol= 'yf_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='PEER_PRESSURE', outputCol= 'pp_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='CHRONIC_DISEASE', outputCol= 'chd_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='WHEEZING', outputCol= 'wh_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='ALCOHOL_CONSUMING', outputCol= 'acm_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='COUGHING', outputCol= 'cgh_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='SHORTNESS_OF_BREATH', outputCol= 'shs_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='SWALLOWING_DIFFICULTY', outputCol= 'shdy_idx').fit(cpulmon).transform(cpulmon)
cpulmon = StringIndexer(inputCol='CHEST_PAIN', outputCol= 'chp_idx').fit(cpulmon).transform(cpulmon)

In [28]:
cpulmon = StringIndexer(inputCol='LUNG_CANCER', outputCol= 'label').fit(cpulmon).transform(cpulmon)


In [30]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['gd_idx','smk_idx','yf_idx','pp_idx','chd_idx', 'wh_idx', 'acm_idx','cgh_idx', 'shs_idx' ,'shdy_idx','chp_idx'],outputCol='features' )
cpulmon= assembler.transform(cpulmon)

In [31]:
cpulmon.select('features').show(10)
cpulmon.select('label').show()

+--------------------+
|            features|
+--------------------+
|(11,[1,3,4,9],[1....|
|(11,[2,3,5,6,7,9]...|
|(11,[0,1,2,4,6],[...|
|(11,[3,4,5,7,8,9]...|
|(11,[0,1,3,4,6,10...|
|(11,[0,1,3,6,10],...|
|(11,[2,3,4],[1.0,...|
|[1.0,0.0,0.0,0.0,...|
|[1.0,0.0,1.0,1.0,...|
|(11,[5,7,8,9],[1....|
+--------------------+
only showing top 10 rows

+-----+
|label|
+-----+
|  0.0|
|  0.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
|  0.0|
|  0.0|
|  1.0|
|  0.0|
|  0.0|
|  0.0|
|  1.0|
|  0.0|
|  1.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  1.0|
+-----+
only showing top 20 rows



In [32]:
cp_train, cp_test = cpulmon.randomSplit([0.8, 0.2], seed=23)
[cp_train.count(), cp_test.count()]

[8379, 2127]

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree_model = tree.fit(cp_train)

In [34]:
prediction = tree_model.transform(cp_test)
prediction.select('label', 'prediction', 'probability').show(5, False)
prediction.groupBy("label", "prediction").count().show()


+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|1.0  |1.0       |[0.39705882352941174,0.6029411764705882] |
|1.0  |1.0       |[0.39705882352941174,0.6029411764705882] |
|1.0  |1.0       |[0.39705882352941174,0.6029411764705882] |
|0.0  |0.0       |[0.9362745098039216,0.06372549019607843] |
|0.0  |0.0       |[0.9713690786048933,0.028630921395106715]|
+-----+----------+-----------------------------------------+
only showing top 5 rows

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  208|
|  0.0|       1.0|   65|
|  1.0|       0.0|   67|
|  0.0|       0.0| 1787|
+-----+----------+-----+



# Evaluación del modelo

In [35]:
#Hallando valores para la matriz de confusion
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()


In [36]:
precision = (TN+ TP)/(TN+TP+FN+FP)
print(precision)


0.9379407616361072


In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

df_accuracy = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy").evaluate(prediction)
df_accuracy

0.9379407616361072

In [38]:
df_precision = MulticlassClassificationEvaluator(labelCol="label", metricName="weightedPrecision").evaluate(prediction)
df_precision

0.9377508630387703